In [1]:
import pandas as pd
import numpy as np
import sys
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col, mean, min, max
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor, RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

%load_ext autotime

In [3]:
#sc = SQLContext()
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
       .map(lambda line: line.split(','))
   #didn’t work with take(1). believe returns
   #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

df = df_from_csv('dave2.csv')

#here’s the new vectorizer function:

def vectorizeData(data):
    return data.rdd.map(lambda r: [r[0], r[1], r[2], Vectors.dense(r[3:6]), float(r[7])]).toDF([\
           'cluster', 'userid', 'workoutid', 'features', 'label'])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 7, 172.31.42.143, executor 0): java.io.FileNotFoundException: File file:/home/ubuntu/dave2.csv/part-00131-43e007d8-f04d-48f3-8777-8390676cb3c3.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:421)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:142)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:346)
	at org.apache.hadoop.fs.FileSystem.op

time: 544 ms


In [4]:
select_columns = ['prediction', 'userid', 'workoutid', 'sum_geo_distance', 'diff_altitude', \
                'new_avg_speed', 'new_avg_dist', 'max_elapsed_time']
df = df.select(select_columns)

NameError: name 'df' is not defined

time: 11.4 ms


In [5]:
df.schema

NameError: name 'df' is not defined

time: 10.2 ms


In [31]:
reg_df = vectorizeData(df)

distinct_clusters = reg_df.select('cluster').distinct().collect()
cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]



time: 2.24 s


In [36]:
reg_df.schema

StructType(List(StructField(cluster,StringType,true),StructField(userid,StringType,true),StructField(workoutid,StringType,true),StructField(features,VectorUDT,true),StructField(label,DoubleType,true)))

time: 2.29 ms


In [1]:
print reg_df[reg_df['cluster']==i]

NameError: name 'reg_df' is not defined

In [16]:
#broken
for i in distinct_clusters:
    print i
    print reg_df[reg_df['cluster']==i].count()

Row(cluster=u'7')


Py4JJavaError: An error occurred while calling o157.equalTo.
: java.lang.RuntimeException: Unsupported literal type class java.util.ArrayList [7]
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:75)
	at org.apache.spark.sql.functions$.lit(functions.scala:101)
	at org.apache.spark.sql.Column.$eq$eq$eq(Column.scala:267)
	at org.apache.spark.sql.Column.equalTo(Column.scala:290)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:280)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:

time: 67.5 ms


In [33]:
cluster_numbers

[7, 3, 8, 0, 5, 6, 2]

time: 6.64 ms


In [23]:
#masa edit
model_list = [LinearRegression(featuresCol="features", labelCol="label"),\
              DecisionTreeRegressor(featuresCol="features", labelCol="label"), \
              GBTRegressor(featuresCol="features", labelCol="label"),\
              RandomForestRegressor(featuresCol="features", labelCol="label")]


distinct_clusters = reg_df.select('cluster').distinct().collect()

cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

for n in model_list:    
    for i in cluster_numbers:
        temp_df = reg_df[reg_df['cluster']==i]
        temp_lr = n
        temp_lrModel = temp_lr.fit(temp_df['label','features'])
        temp_df = temp_lrModel.transform(temp_df)
        
        #paramGrid = ParamGridBuilder() \
        #.addGrid(hashingTF.numFeatures, [10, 100, 1000]) \
        #.addGrid(lr.regParam, [0.1, 0.01]) \
        #.build()

        
        
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        
        #crossval = CrossValidator(estimator=pipeline,
        #                          estimatorParamMaps=paramGrid,
        #                          evaluator=BinaryClassificationEvaluator(),
        #                          numFolds=3)  # use 3+ folds in practice#

        # Run cross-validation, and choose the best set of parameters.
        #cvModel = crossval.fit(training)

        #pred = cvModel.transform(temp_df)
        
        print("For " + str(type(temp_lr)) + " and cluster " + str(i))
        #Print the coefficients and intercept for linear regression
        #print("Coefficients: " + str(temp_lrModel.coefficients))
        #print("Intercept: " + str(temp_lrModel.intercept))
        
        
       
        rmse = evaluator.evaluate(temp_df)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


For <class 'pyspark.ml.regression.LinearRegression'> and cluster 7
Root Mean Squared Error (RMSE) on test data = 277.277
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 3
Root Mean Squared Error (RMSE) on test data = 40.1564
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 8
Root Mean Squared Error (RMSE) on test data = 638.485
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 0
Root Mean Squared Error (RMSE) on test data = 562.182
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 5
Root Mean Squared Error (RMSE) on test data = 823.759
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 6
Root Mean Squared Error (RMSE) on test data = 197.481
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 2
Root Mean Squared Error (RMSE) on test data = 444.264
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 7
Root Mean Squared Error (RMSE) on test data = 0.99637
For <class 'pyspark.ml.regr

In [25]:
#LinearRegression
#masa edit w/ CV
model_list = [LinearRegression(featuresCol="features", labelCol="label")]#,\
              #DecisionTreeRegressor(featuresCol="features", labelCol="label"), \
              #GBTRegressor(featuresCol="features", labelCol="label"),\
              #RandomForestRegressor(featuresCol="features", labelCol="label")]


distinct_clusters = reg_df.select('cluster').distinct().collect()

cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

for n in model_list:    
    for i in cluster_numbers:
        temp_df = reg_df[reg_df['cluster']==i]
        temp_df_cv = temp_df
        temp_lr = n
        temp_lrModel = temp_lr.fit(temp_df['label','features'])
        temp_df = temp_lrModel.transform(temp_df)
        
        paramGrid = ParamGridBuilder() \
        .addGrid(temp_lr.maxIter, [5, 10, 100]) \
        .addGrid(temp_lr.regParam, [0, 0.1, 0.01]) \
        .build()

        
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        
        crossval = CrossValidator(estimator=temp_lr,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=2)  # use 3+ folds in practice

        # Run cross-validation, and choose the best set of parameters.
        cvModel = crossval.fit(temp_df_cv)

        pred = cvModel.transform(temp_df_cv)
        
        print("For " + str(type(temp_lr)) + " and cluster " + str(i))
        #Print the coefficients and intercept for linear regression
        #print("Coefficients: " + str(temp_lrModel.coefficients))
        #print("Intercept: " + str(temp_lrModel.intercept))
        
        

        rmse = evaluator.evaluate(pred)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


For <class 'pyspark.ml.regression.LinearRegression'> and cluster 7
Root Mean Squared Error (RMSE) on test data = 277.277
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 3
Root Mean Squared Error (RMSE) on test data = 40.1566
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 8
Root Mean Squared Error (RMSE) on test data = 638.485
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 0
Root Mean Squared Error (RMSE) on test data = 562.182
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 5
Root Mean Squared Error (RMSE) on test data = 823.759
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 6
Root Mean Squared Error (RMSE) on test data = 197.481
For <class 'pyspark.ml.regression.LinearRegression'> and cluster 2
Root Mean Squared Error (RMSE) on test data = 444.264
time: 26min 9s


In [19]:
#DecisionTreeRegressor
#masa edit w/ CV
#[LinearRegression(featuresCol="features", labelCol="label"),\
model_list = [DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=2056)]#, \
              #GBTRegressor(featuresCol="features", labelCol="label"),\
              #RandomForestRegressor(featuresCol="features", labelCol="label")]


distinct_clusters = reg_df.select('cluster').distinct().collect()

cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

for n in model_list:    
    for i in cluster_numbers:
        temp_df = reg_df[reg_df['cluster']==i]
        temp_df_cv = temp_df
        temp_lr = n
        temp_lrModel = temp_lr.fit(temp_df['label','features'])
        temp_df = temp_lrModel.transform(temp_df)
        
        paramGrid = ParamGridBuilder() \
        .addGrid(temp_lr.maxDepth, [3, 5]) \
        .addGrid(temp_lr.minInfoGain, [0, 0.1, 1]) \
        .build()

        
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        
        crossval = CrossValidator(estimator=temp_lr,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=2)  # use 3+ folds in practice

        # Run cross-validation, and choose the best set of parameters.
        cvModel = crossval.fit(temp_df_cv)

        pred = cvModel.transform(temp_df_cv)
        
        
        print("For " + str(type(temp_lr)) + " and cluster " + str(i))
        #print(cvModel.explainParams())

        #Print the coefficients and intercept for linear regression
        #print("Coefficients: " + str(temp_lrModel.coefficients))
        #print("Intercept: " + str(temp_lrModel.intercept))
        
        

        rmse = evaluator.evaluate(pred)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 7
Root Mean Squared Error (RMSE) on test data = 154.807
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 3
Root Mean Squared Error (RMSE) on test data = 1.44338
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 8
Root Mean Squared Error (RMSE) on test data = 0
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 0
Root Mean Squared Error (RMSE) on test data = 263.248
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 5
Root Mean Squared Error (RMSE) on test data = 252.745
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 6
Root Mean Squared Error (RMSE) on test data = 0
For <class 'pyspark.ml.regression.DecisionTreeRegressor'> and cluster 2
Root Mean Squared Error (RMSE) on test data = 324.623
time: 15min 32s


In [34]:
#Gradient Boosted Trees
#masa edit w/ CV
#[LinearRegression(featuresCol="features", labelCol="label"),\
#DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=1028)]#, \             
model_list = [GBTRegressor(featuresCol="features", labelCol="label", maxMemoryInMB=2056)]#,\
              #RandomForestRegressor(featuresCol="features", labelCol="label")]


distinct_clusters = reg_df.select('cluster').distinct().collect()

cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

for n in model_list:    
    for i in cluster_numbers:
        temp_df = reg_df[reg_df['cluster']==i]
        temp_df_cv = temp_df
        temp_lr = n
        temp_lrModel = temp_lr.fit(temp_df['label','features'])
        temp_df = temp_lrModel.transform(temp_df)
        
        paramGrid = ParamGridBuilder() \
        .addGrid(temp_lr.maxDepth, [3, 5]) \
        .addGrid(temp_lr.maxIter, [10,20,40]) \
        .build()

        
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        
        crossval = CrossValidator(estimator=temp_lr,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=2)  # use 3+ folds in practice

        # Run cross-validation, and choose the best set of parameters.
        cvModel = crossval.fit(temp_df_cv)

        pred = cvModel.transform(temp_df_cv)
        
        
        print("For " + str(type(temp_lr)) + " and cluster " + str(i))
        #print(cvModel.explainParams())

        #Print the coefficients and intercept for linear regression
        #print("Coefficients: " + str(temp_lrModel.coefficients))
        #print("Intercept: " + str(temp_lrModel.intercept))
        
        

        rmse = evaluator.evaluate(pred)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 7
Root Mean Squared Error (RMSE) on test data = 28.0001
For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 3
Root Mean Squared Error (RMSE) on test data = 0.193727
For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 8
Root Mean Squared Error (RMSE) on test data = 0
For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 0
Root Mean Squared Error (RMSE) on test data = 145.209
For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 5
Root Mean Squared Error (RMSE) on test data = 3.49461
For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 6
Root Mean Squared Error (RMSE) on test data = 0
For <class 'pyspark.ml.regression.GBTRegressor'> and cluster 2
Root Mean Squared Error (RMSE) on test data = 223.04
time: 33min 59s


In [35]:
#Random Forest Regressor
#masa edit w/ CV
#[LinearRegression(featuresCol="features", labelCol="label"),\
#DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=1028)], \             
#GBTRegressor(featuresCol="features", labelCol="label", maxMemoryInMB=2056)],\
model_list = [RandomForestRegressor(featuresCol="features", labelCol="label", maxMemoryInMB = 2056)]


distinct_clusters = reg_df.select('cluster').distinct().collect()

cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

for n in model_list:    
    for i in cluster_numbers:
        temp_df = reg_df[reg_df['cluster']==i]
        temp_df_cv = temp_df
        temp_lr = n
        temp_lrModel = temp_lr.fit(temp_df['label','features'])
        temp_df = temp_lrModel.transform(temp_df)
        
        paramGrid = ParamGridBuilder() \
        .addGrid(temp_lr.maxDepth, [3, 5]) \
        .addGrid(temp_lr.numTrees, [10,20,40]) \
        .build()

        
        evaluator = RegressionEvaluator(
            labelCol="label", predictionCol="prediction", metricName="rmse")
        
        crossval = CrossValidator(estimator=temp_lr,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=evaluator,
                                  numFolds=2)  # use 3+ folds in practice

        # Run cross-validation, and choose the best set of parameters.
        cvModel = crossval.fit(temp_df_cv)

        pred = cvModel.transform(temp_df_cv)
        
        
        print("For " + str(type(temp_lr)) + " and cluster " + str(i))
        #print(cvModel.explainParams())

        #Print the coefficients and intercept for linear regression
        #print("Coefficients: " + str(temp_lrModel.coefficients))
        #print("Intercept: " + str(temp_lrModel.intercept))
        
        

        rmse = evaluator.evaluate(pred)
        print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 7
Root Mean Squared Error (RMSE) on test data = 144.709
For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 3
Root Mean Squared Error (RMSE) on test data = 92.0182
For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 8
Root Mean Squared Error (RMSE) on test data = 526.841
For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 0
Root Mean Squared Error (RMSE) on test data = 261.819
For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 5
Root Mean Squared Error (RMSE) on test data = 632.921
For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 6
Root Mean Squared Error (RMSE) on test data = 89.2734
For <class 'pyspark.ml.regression.RandomForestRegressor'> and cluster 2
Root Mean Squared Error (RMSE) on test data = 353.878
time: 17min 48s
